In [1]:
import numpy as np
import pandas as pd
from econml.grf import CausalForest, CausalIVForest, RegressionForest
from econml.dml import CausalForestDML
import scipy.special
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

c:\Users\Krishna\anaconda3\envs\gen_rfm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url_1 = r"https://www.dropbox.com/s/up0k1i6btb8qmb7/usa_00004.csv?dl=1"
#url_1 = r"C:\Users\Krishna\Dropbox\usa_00004.csv"
data_2019 = pd.read_csv(url_1)
data_2019.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2019,201901,1,2019010000088,11.0,2019000000011,0,1,220001,4,...,6,2,200,4,40,3,30,9000,NaN,NaN
1,2019,201901,2,2019010000096,70.0,2019000000021,0,1,100001,3,...,6,1,100,4,40,3,30,150,NaN,NaN
2,2019,201901,3,2019010000153,20.0,2019000000031,15,4,110001,4,...,6,2,200,7,71,1,10,1400,NaN,NaN
3,2019,201901,4,2019010000198,79.0,2019000000041,15,4,110001,3,...,5,1,100,3,30,3,30,22700,NaN,NaN
4,2019,201901,5,2019010000205,53.0,2019000000051,97,4,270101,3,...,3,2,200,3,30,3,30,0,NaN,NaN


In [3]:
#url_2 = r"https://www.dropbox.com/s/yep1ue5e4u2qhc3/usa_00005.csv?dl=1"
url_2 = r"C:\Users\Krishna\Dropbox\usa_00005.csv"
data_2015 = pd.read_csv(url_2)
data_2015.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,MARST,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,5,50,1,10,3450,2.0,61.0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,1,2,200,7,71,3,30,14000,1.0,56.0
2,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,6,2,200,1,15,0,0,9999999,NaN,NaN
3,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,4,2,200,7,71,1,10,35000,NaN,NaN
4,2015,201501,2,160,65.0,2015000000021,0,1,100001,1,...,4,8,802,2,26,3,30,8800,NaN,NaN


In [4]:
data_2015.columns


Index(['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'COUNTYFIP',
       'METRO', 'STRATA', 'GQ', 'SSMC', 'PERNUM', 'PERWT', 'SEX', 'AGE',
       'MARST', 'RACE', 'RACED', 'EDUC', 'EDUCD', 'EMPSTAT', 'EMPSTATD',
       'INCTOT', 'SEX_SP', 'AGE_SP'],
      dtype='object')

In [5]:
print(data_2015.shape)
data_2015 = data_2015.dropna()
print(data_2015.shape)

(3147005, 25)
(1419246, 25)


In [6]:
data_2015['SEX_SP'] = data_2015['SEX_SP'].astype(int)

data_2015.loc[:, 'same_sex_couple'] = 0
data_2015.loc[data_2015['SEX'] == data_2015['SEX_SP'], 'same_sex_couple'] = 1

In [7]:

check_data = data_2015[['SEX',
                        'SEX_SP',
                        'same_sex_couple']].copy()
check_data

,SEX,SEX_SP,same_sex_couple
0,1,2,0
1,2,1,0
6,1,2,0
7,2,1,0
11,2,1,0
...,...,...,...
3146985,2,1,0
3146988,1,2,0
3146989,2,1,0
3146996,2,1,0


In [8]:
data_2015.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,RACE,RACED,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP,same_sex_couple
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,2,200,5,50,1,10,3450,2,61.0,0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,2,200,7,71,3,30,14000,1,56.0,0
6,2015,201501,4,463,36.0,2015000000041,97,4,270101,1,...,1,100,10,101,1,10,150000,2,53.0,0
7,2015,201501,4,463,36.0,2015000000041,97,4,270101,1,...,1,100,7,71,1,10,6000,1,53.0,0
11,2015,201501,6,518,175.0,2015000000061,0,0,10001,1,...,1,100,0,2,3,30,0,1,53.0,0


In [9]:
data_2015['same_sex_couple'].value_counts()

0    1398684
1      20562
Name: same_sex_couple, dtype: int64

In [10]:
data_2019 = data_2019.dropna()

In [11]:
data_2019['SEX_SP'] = data_2019['SEX_SP'].astype(int)

data_2019.loc[:, 'same_sex_couple'] = 0
data_2019.loc[data_2019['SEX'] == data_2019['SEX_SP'], 'same_sex_couple'] = 1
data_2019['same_sex_couple'].unique()



array([0, 1], dtype=int64)

In [12]:
check_data1 = data_2019[['SEX',
                        'SEX_SP',
                        'same_sex_couple']].copy()
check_data1
check_data1.dtypes

SEX                int64
SEX_SP             int32
same_sex_couple    int64
dtype: object

In [13]:
data_2019["MARST"].value_counts()

1    1338592
6      91512
4      50395
5       6480
3       4125
Name: MARST, dtype: int64

In [14]:
data_2015.loc[:, 'married_couple'] = 0
data_2015.loc[data_2015['MARST'] <= 1, 'married_couple'] = 1
data_2015["married_couple"].value_counts()

1    1282842
0     136404
Name: married_couple, dtype: int64

In [15]:
data_2019.loc[:, 'married_couple'] = 0
data_2019.loc[data_2019['MARST'] <= 1, 'married_couple'] = 1
data_2019["married_couple"].value_counts()

1    1338592
0     152512
Name: married_couple, dtype: int64

In [21]:
#concat dataset
frames = [data_2015, data_2019]
final_df = pd.concat(frames)
final_df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,COUNTYFIP,METRO,STRATA,GQ,...,EDUC,EDUCD,EMPSTAT,EMPSTATD,INCTOT,SEX_SP,AGE_SP,same_sex_couple,married_couple,treat
0,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,5,50,1,10,3450,2,61.0,0,1,0.0
1,2015,201501,1,67,48.0,2015000000011,97,4,270101,1,...,7,71,3,30,14000,1,56.0,0,1,0.0
6,2015,201501,4,463,36.0,2015000000041,97,4,270101,1,...,10,101,1,10,150000,2,53.0,0,1,0.0
7,2015,201501,4,463,36.0,2015000000041,97,4,270101,1,...,7,71,1,10,6000,1,53.0,0,1,0.0
11,2015,201501,6,518,175.0,2015000000061,0,0,10001,1,...,0,2,3,30,0,1,53.0,0,1,0.0


In [22]:
final_df["YEAR"].unique()

array([2015, 2019], dtype=int64)

In [17]:
features = data_2015[['SEX', 'AGE', 'RACE', 'EDUC', 'EMPSTAT']]
treatment = data_2015['married_couple']
instrument = data_2015['INCTOT']

In [18]:
data_2015['treat'] = data_2015.apply(lambda _:0, axis = 1)



In [19]:
np.random.seed(1227)
n_samples = 2000
n_features = len(features.columns)
n_treatments = 1
# true_te = lambda X: np.hstack([X[:, [0]]**2 + 1, np.ones((X.shape[0], n_treatments - 1))])
true_te = lambda X: np.hstack([X[:, [0]]>0, np.ones((X.shape[0], n_treatments - 1))])
# true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
#                                np.ones((X.shape[0], n_treatments - 1))*np.arange(1, n_treatments).reshape(1, -1)])
Z = np.random.normal(0, 1, size=(n_samples, n_treatments))
X = np.random.normal(0, 1, size=(n_samples, n_features))
U = np.random.normal(0, .2, size=(n_samples, 1))
T = np.random.normal(0, 1, size=(n_samples, n_treatments))
for t in range(n_treatments):
    T[:, t] += U[:, 0] + Z[:, t]
y = np.sum(true_te(X) * T, axis=1, keepdims=True) + 10 * U[:, [0]]
X_test = X[:1000].copy()
X_test[:, 0] = np.linspace(np.percentile(X[:, 0], 1), np.percentile(X[:, 0], 99), 1000)